In [31]:
import os
import pandas as pd


directory = r"C:\Users\rachitgandhi1\OneDrive - KPMG\Desktop\Data"
#input('Please enter the folder directory path :')  # Replace with the actual directory path
#year_repo = input('Please enter the year :')

# List all files in the directory
files = os.listdir(directory)

turbine1_df = pd.DataFrame()
turbine2_df = pd.DataFrame()
turbine3_df = pd.DataFrame()
turbine4_df = pd.DataFrame()
turbine5_df = pd.DataFrame()
turbine6_df = pd.DataFrame()

turbines = [turbine1_df, turbine2_df, turbine3_df,turbine4_df, turbine5_df, turbine6_df ]


csv_1_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_1')]
csv_2_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_2')]
csv_3_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_3')]
csv_4_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_4')]
csv_5_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_5')]
csv_6_files = [file for file in files if file.endswith('.csv') and file.startswith('Turbine_Data_Kelmarsh_6')]

csv_i_files  = [csv_1_files,csv_2_files,csv_3_files,csv_4_files,csv_5_files,csv_6_files]


In [32]:
turbine1_df = pd.DataFrame()
for csv_1_file in csv_1_files:
    path = r'C:\Users\rachitgandhi1\OneDrive - KPMG\Desktop\Data'
    df = pd.read_csv(f'{path}\{csv_1_file}',skiprows=8, header=1)
    print(df.size)
    turbine1_df = pd.concat([turbine1_df,df], axis=0)
    print(turbine1_df.size)
    

15715440
15715440
15758496
31473936
7793136
39267072


In [33]:
import data_preprocess as dpf
turbine1_df_fi = dpf.data_process(turbine1_df)

turbine1_df_final = dpf.normalize_all(turbine1_df_fi)

In [34]:
# calculate the median of each column over the last 14 days, ignoring NaN values
# Assuming 'df' is your DataFrame
turbine1_df_final = turbine1_df_final.dropna(subset=['Wind speed (m/s)'])
# Assuming 'df' is your DataFrame and the columns are ['column1', 'column2', 'column3']

turbine1_df_final['Metal particle count counter'].fillna(method='ffill', inplace=True)
turbine1_df_final['Tower Acceleration y (mm/ss)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Tower Acceleration X (mm/ss)'].fillna(method='ffill', inplace=True)
turbine1_df_final['blade_angle'].fillna(method='ffill', inplace=True)
turbine1_df_final['Front bearing temperature (°C)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Rear bearing temperature (°C)'].fillna(method='ffill', inplace=True)
turbine1_df_final['Nacelle ambient temperature (°C)'].fillna(method='ffill', inplace=True)

for i in (1,50):
    means = turbine1_df_final.rolling(window=7, min_periods=1).mean(numeric_only=True)

    # replace NaN values in each column with the corresponding median value, or 0 if there are no non-NaN values
    turbine1_df_final = turbine1_df_final.fillna(means)





C:\Users\rachitgandhi1\AppData\Local\Temp\ipykernel_20992\1273828923.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turbine1_df_final['Metal particle count counter'].fillna(method='ffill', inplace=True)
C:\Users\rachitgandhi1\AppData\Local\Temp\ipykernel_20992\1273828923.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  turbine1_df_final['Tower Acceleration y (mm/ss)'].fillna(method='ffill', inplace=True)
C:\Users\rachitgandhi1\AppData\Local\Temp\ipykernel_20992\1273828923.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https

In [35]:
turbine1_df_final = turbine1_df_final.reset_index()


In [36]:
import pandas as pd


# specify the list of features to lag
diff_features = ['blade_angle','Nacelle ambient temperature (°C)','Tower Acceleration X (mm/ss)','Tower Acceleration y (mm/ss)']
features_to_lag = ['Wind direction (°)','Rear bearing temperature (°C)',
                'Rotor speed (RPM)','Generator RPM (RPM)',
                'Front bearing temperature (°C)','Tower Acceleration X (mm/ss)','Wind speed (m/s)','Tower Acceleration y (mm/ss)','Metal particle count counter']
print("Start")
# lag the specified features by 1, 2, 3, 4, and 5 timesteps
for feature in features_to_lag:
    for i in range(1, 6):
        feature_n = f'{feature}_lagged_{i}'
        turbine1_df_final[feature_n] = turbine1_df_final[feature].shift(i)

for feature in diff_features:
    feature_n = f'{feature}_diff'
    turbine1_df_final[feature_n] = turbine1_df_final[feature].diff()
               

# fill missing values with the previous value
turbine1_df_final.fillna(method='ffill', inplace=True)


Start


In [37]:
turbine1_df_final.fillna(0, inplace=True)



In [38]:
print(turbine1_df_final.isnull().sum())

index                                    0
# Date and time                          0
Wind speed (m/s)                         0
Wind direction (°)                       0
Nacelle position (°)                     0
                                        ..
Metal particle count counter_lagged_5    0
blade_angle_diff                         0
Nacelle ambient temperature (°C)_diff    0
Tower Acceleration X (mm/ss)_diff        0
Tower Acceleration y (mm/ss)_diff        0
Length: 64, dtype: int64


In [42]:
turbine1_df_final_numeric = turbine1_df_final.select_dtypes(include='int')
turbine1_df_final_numeric.to_csv('nan.csv', na_rep='nan', index=False)



In [40]:
turbine1_df_final.to_csv('input_data.csv', index_label=False)

Start
        index      # Date and time  Wind speed (m/s)  Wind direction (°)  \
0           0  2019-01-01 00:00:00          0.193842            0.790682   
1           1  2019-01-01 00:10:00          0.188339            0.802161   
2           2  2019-01-01 00:20:00          0.238407            0.811375   
3           3  2019-01-01 00:30:00          0.239560            0.818464   
4           4  2019-01-01 00:40:00          0.245320            0.830623   
...       ...                  ...               ...                 ...   
130723  26059  2021-06-30 23:10:00          0.166287            0.110955   
130724  26060  2021-06-30 23:20:00          0.179300            0.110289   
130725  26061  2021-06-30 23:30:00          0.163595            0.117261   
130726  26062  2021-06-30 23:40:00          0.080132            0.115900   
130727  26063  2021-06-30 23:50:00          0.084619            0.094233   

        Nacelle position (°)  Power (kW)  Front bearing temperature (°C)  \
0                   0.769874    0.099892                        0.833237   
1                   0.769874    0.098801                        0.811152   
2                   0.769874    0.190098                        0.854967   
3                   0.769874    0.209905                        0.898783   
4                   0.822596    0.155019                        0.885980   
...                      ...         ...                             ...   
130723              0.096058    0.032630                        0.769861   
130724              0.096058    0.054875                        0.769861   
130725              0.096058    0.043631                        0.769861   
130726              0.096058    0.004039                        0.769861   
130727              0.096058    0.007984                        0.769861   
...
130726                           0.000000                           0.000000  
130727                           0.000000                           0.000000  

[130728 rows x 79 columns]
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...
Index(['index', '# Date and time', 'Wind speed (m/s)', 'Wind direction (°)',
       'Nacelle position (°)', 'Power (kW)', 'Front bearing temperature (°C)',
       'Rear bearing temperature (°C)', 'Nacelle ambient temperature (°C)',
       'Rotor speed (RPM)', 'Generator RPM (RPM)',
       'Metal particle count counter', 'Tower Acceleration X (mm/ss)',
       'Tower Acceleration y (mm/ss)', 'blade_angle',
       'Wind direction (°)_lagged_1', 'Wind direction (°)_lagged_2',
       'Wind direction (°)_lagged_3', 'Wind direction (°)_lagged_4',
       'Wind direction (°)_lagged_5', 'Nacelle position (°)_lagged_1',
       'Nacelle position (°)_lagged_2', 'Nacelle position (°)_lagged_3',
       'Nacelle position (°)_lagged_4', 'Nacelle position (°)_lagged_5',
       'blade_angle_lagged_1', 'blade_angle_lagged_2', 'blade_angle_lagged_3',
       'blade_angle_lagged_4', 'blade_angle_lagged_5',
       'Rear bearing temperature (°C)_lagged_1',
       'Rear bearing temperature (°C)_lagged_2',
       'Rear bearing temperature (°C)_lagged_3',
       'Rear bearing temperature (°C)_lagged_4',
       'Rear bearing temperature (°C)_lagged_5', 'Rotor speed (RPM)_lagged_1',
       'Rotor speed (RPM)_lagged_2', 'Rotor speed (RPM)_lagged_3',
       'Rotor speed (RPM)_lagged_4', 'Rotor speed (RPM)_lagged_5',
       'Generator RPM (RPM)_lagged_1', 'Generator RPM (RPM)_lagged_2',
       'Generator RPM (RPM)_lagged_3', 'Generator RPM (RPM)_lagged_4',
       'Generator RPM (RPM)_lagged_5',
       'Nacelle ambient temperature (°C)_lagged_1',
       'Nacelle ambient temperature (°C)_lagged_2',
...
       'Nacelle ambient temperature (°C)_diff',
       'Tower Acceleration X (mm/ss)_diff',
       'Tower Acceleration y (mm/ss)_diff'],
      dtype='object')
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...